In [1]:
# Load kedro environment (not needed in .py)
from pathlib import Path
from kedro.framework.context import load_context

# Load a context to be able to work in the notebook
#current_dir = Path.cwd()
current_dir = Path("/u01/share/cesar/aa_engine_uy/notebooks/")
proj_path = current_dir.parent
context = load_context(proj_path)
catalog = context.catalog
credentials = context.config_loader.get("credentials*","credentials*/**")
parameters = context.config_loader.get("parameters*","parameters*/**")

from aa_engine_pkg.assets.utils import *
from aa_engine_pkg.assets.core.data.kedro.catalog_expansion.partitioned_sql import SQLPartitionedDataSet

In [2]:
date='20180605'

In [3]:
def create_cliente_activo(cliente_activo: SQLPartitionedDataSet,
                          date: str) -> pd.DataFrame:
    """Creates master table with features related to EoP state of customers for one period of data

    Parameters
    ----------
    cliente_activo:
        dataset defined in ´catalog.yml´ - list of active customers at EoP for the given period
    date:
        period to process
    
    Returns
    -------
        Mastertable with information of clientes at EoP
    """

    # Initialize logger
    log = initialize_logger()

    # Load active clientes for period
    log.info(f"Creating cliente_activo...")
    period_to_load = get_previous_month(date)
    df_clientes_activos = cliente_activo.filter_by(date=period_to_load)

    # Return
    return df_clientes_activos

In [4]:
cliente_activo=catalog.load("cliente_activo")

2021-01-19 18:39:50,249 - kedro.io.data_catalog - INFO - Loading data from `cliente_activo` (SQLPartitionedDataSet)...


In [5]:
cliente_activo_df= create_cliente_activo(cliente_activo,date)

2021-01-19 18:39:51,200 - aa_engine_pkg.assets.utils.utilities - INFO - Creating cliente_activo...
select distinct CUSTOMER_ID from stg_uy_customer_status where UPPER(STATUS) LIKE '%ACTIVO%' and DATE_EXP = 201805


/u01/miniconda3/envs/uypo37/lib/python3.7/site-packages/sqlalchemy/dialects/oracle/base.py:1381: SAWarning: Oracle version (19, 5, 0, 0, 0) is known to have a maximum identifier length of 128, rather than the historical default of 30. SQLAlchemy 1.4 will use 128 for this database; please set max_identifier_length=128 in create_engine() in order to test the application with this new length, or set to 30 in order to assure that 30 continues to be used.  In particular, pay close attention to the behavior of database migrations as dynamically generated names may change. See the section 'Max Identifier Lengths' in the SQLAlchemy Oracle dialect documentation for background.
  % ((self.server_version_info,))


In [6]:
servicioalcliente=catalog.load("servicioalcliente")

2021-01-19 18:39:54,280 - kedro.io.data_catalog - INFO - Loading data from `servicioalcliente` (SQLPartitionedDataSet)...


In [7]:
string_vars = ["CATEGORY",
               "SUB_CATEGORY",
               "SECOND_SUB_CATEGORY",
               "METHOD"]

vars_to_group_by = ["CUSTOMER_ID", "FECHA_CREATE"]

id_cols = ["CUSTOMER_ID", "DATE_EXP", "DATE_CALC"]

categories_dict = {"CATEGORY_COMBINED": ['0102',	'0103',	'0105',	'0108',	'0109',	'0110',	'0111',	'0112',	
                                         '0205',	'0206',
                                         '0301',
                                         '0401',	'0403',
                                         '0601',	'0605',
                                         '0702',
                                         '0801',	'0802',	'0803',	'0805',
                                         '1003',
                                         '1101',	'1105',	'1110',	'1111',
                                         '1201',	'1202',	'1203',	'1204',	'1205',	'1205',	'1206',	'1207',	'1208',
                                         '1302'],
                   
                   "SUB_CATEGORY_COMBINED": ['0102001',	'0102002',	'0103001',	'0105001',	'0105002',	'0108001',	'0108006',	'0109001',	'0111003',	'0112001',					
                                             '0205001',	'0206005',													
                                             '0702007',														
                                             '0802004',	'0805002',	'0805003',												
                                             '1003004',														
                                             '1101003',	'1105003',	'1111001',												
                                             '1201003',	'1201007',	'1202048',	'1203018',	'1203019',
                                             '1203020',	'1203021',	'1205028',	'1205029',	'1205030',	'1205049',	'1206032',	'1207042',	'1208044','1208047'],
                   "CHANNEL_DIR": None,
                   "CHANNEL": None}

past_periods = [7, 28, 84, 168]


In [8]:
# Initialize logger
log = initialize_logger()
log.info("Creating master table servicio al cliente")

# Read parameters
lookback_days = parameters["masters"]["global"]["look_back_days"]
start_date = (pd.to_datetime(date) - timedelta(days=lookback_days)).strftime("%Y%m%d")

# Calculate period to load for active clients
log.info("Loading table clientes")
period_to_load = get_previous_month(date)
df_clientes = cliente_activo_df

log.info("Loading table servicioalcliente")
df_sac = servicioalcliente.filter_by(date=[start_date,
                                           date]).drop_duplicates()

df_sac = pd.merge(
    df_clientes,
    df_sac,
    on=["CUSTOMER_ID"],
    validate="1:m"
)

# Fill in rows with blank CATEGORY 
mask_blank_category = df_sac.CATEGORY.isna()
df_sac.loc[mask_blank_category, "CATEGORY"] = df_sac.loc[mask_blank_category, "SUB_CATEGORY"]
df_sac.loc[mask_blank_category, "SUB_CATEGORY"] = df_sac.loc[mask_blank_category, "SECOND_SUB_CATEGORY"]
df_sac.loc[mask_blank_category, "SECOND_SUB_CATEGORY"] = np.NaN

2021-01-19 18:39:55,611 - aa_engine_pkg.assets.utils.utilities - INFO - Creating master table servicio al cliente
2021-01-19 18:39:55,613 - aa_engine_pkg.assets.utils.utilities - INFO - Loading table clientes
2021-01-19 18:39:55,614 - aa_engine_pkg.assets.utils.utilities - INFO - Loading table servicioalcliente
select CUSTOMER_ID, FECHA_CREATE, CATEGORY, SUB_CATEGORY, SECOND_SUB_CATEGORY, METHOD from stg_uy_servicioalcliente where CLASS_ID = 2 and FECHA_CREATE >= to_date('20171219', 'yyyymmdd') and FECHA_CREATE < to_date('20180605', 'yyyymmdd')


In [9]:
df_sac.head()

,CUSTOMER_ID,FECHA_CREATE,CATEGORY,SUB_CATEGORY,SECOND_SUB_CATEGORY,METHOD
0,144472,2018-05-31,12-Optimizacion de Contactos,01-No Entiende/Error Factura,003-Valores facturados/Detalle/ Dise?o,Visita
1,144472,2018-05-07,12-Optimizacion de Contactos,08-Otros,044-Cambio datos de la cuenta/forma de pago,Visita
2,144472,2018-05-07,04-Finanzas,01-Cobranzas,003-Cancela metodo de pago,Fax
3,145360,2018-02-24,12-Optimizacion de Contactos,03-Sin Se?al,021-Fallas origen de transmision/sistemas-apoyo,SMS In
4,145360,2018-02-24,12-Optimizacion de Contactos,03-Sin Se?al,021-Fallas origen de transmision/sistemas-apoyo,Telefono IN


In [10]:
# Format variables
log.info("Extracting category codes...")
vars_to_dummy = []
for var in string_vars:
    log.info(f"--- {var}")
    if var == "METHOD":
        format_string_variable(df_sac, var)
    else:
        new_name = var + "_ID"
        vars_to_dummy.append(new_name)
        df_sac[new_name] = df_sac[var].str.extract(r"([a-zA-Z]*[\d]*)[ ]*\-")

2021-01-19 18:40:17,078 - aa_engine_pkg.assets.utils.utilities - INFO - Extracting category codes...
2021-01-19 18:40:17,080 - aa_engine_pkg.assets.utils.utilities - INFO - --- CATEGORY
2021-01-19 18:40:19,838 - aa_engine_pkg.assets.utils.utilities - INFO - --- SUB_CATEGORY
2021-01-19 18:40:22,261 - aa_engine_pkg.assets.utils.utilities - INFO - --- SECOND_SUB_CATEGORY
2021-01-19 18:40:24,760 - aa_engine_pkg.assets.utils.utilities - INFO - --- METHOD


In [11]:
df_sac.head()

,CUSTOMER_ID,FECHA_CREATE,CATEGORY,SUB_CATEGORY,SECOND_SUB_CATEGORY,METHOD,CATEGORY_ID,SUB_CATEGORY_ID,SECOND_SUB_CATEGORY_ID
0,144472,2018-05-31,12-Optimizacion de Contactos,01-No Entiende/Error Factura,003-Valores facturados/Detalle/ Dise?o,VISITA,12,01,003
1,144472,2018-05-07,12-Optimizacion de Contactos,08-Otros,044-Cambio datos de la cuenta/forma de pago,VISITA,12,08,044
2,144472,2018-05-07,04-Finanzas,01-Cobranzas,003-Cancela metodo de pago,FAX,04,01,003
3,145360,2018-02-24,12-Optimizacion de Contactos,03-Sin Se?al,021-Fallas origen de transmision/sistemas-apoyo,SMS_IN,12,03,021
4,145360,2018-02-24,12-Optimizacion de Contactos,03-Sin Se?al,021-Fallas origen de transmision/sistemas-apoyo,TELEFONO_IN,12,03,021


In [12]:
# Fill Second_sub_category_id 
mask_ID_na = df_sac.SECOND_SUB_CATEGORY_ID.isna()
condlist = [df_sac[mask_ID_na].SECOND_SUB_CATEGORY == "Contactado",
            df_sac[mask_ID_na].SECOND_SUB_CATEGORY == "No contactado"]
choicelist = ["C", "NC"]
df_sac.loc[mask_ID_na, "SECOND_SUB_CATEGORY_ID"] = np.select(condlist, choicelist)

In [13]:
df_sac.head()

,CUSTOMER_ID,FECHA_CREATE,CATEGORY,SUB_CATEGORY,SECOND_SUB_CATEGORY,METHOD,CATEGORY_ID,SUB_CATEGORY_ID,SECOND_SUB_CATEGORY_ID
0,144472,2018-05-31,12-Optimizacion de Contactos,01-No Entiende/Error Factura,003-Valores facturados/Detalle/ Dise?o,VISITA,12,01,003
1,144472,2018-05-07,12-Optimizacion de Contactos,08-Otros,044-Cambio datos de la cuenta/forma de pago,VISITA,12,08,044
2,144472,2018-05-07,04-Finanzas,01-Cobranzas,003-Cancela metodo de pago,FAX,04,01,003
3,145360,2018-02-24,12-Optimizacion de Contactos,03-Sin Se?al,021-Fallas origen de transmision/sistemas-apoyo,SMS_IN,12,03,021
4,145360,2018-02-24,12-Optimizacion de Contactos,03-Sin Se?al,021-Fallas origen de transmision/sistemas-apoyo,TELEFONO_IN,12,03,021


In [14]:
mask_ID_na.sum()

0

In [15]:
# Create compound key for category of contact
df_sac["CATEGORY_COMBINED"] = df_sac["CATEGORY_ID"] + df_sac["SUB_CATEGORY_ID"]
df_sac["SUB_CATEGORY_COMBINED"] = df_sac["CATEGORY_ID"] + df_sac["SUB_CATEGORY_ID"] + df_sac[
    "SECOND_SUB_CATEGORY_ID"]

2021-01-19 18:40:27,294 - numexpr.utils - INFO - Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-01-19 18:40:27,295 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [16]:
# Generate features for contact channel
condlist = [df_sac["METHOD"].str.contains("WEB|WHATS|SOCIAL|MAIL|CHAT|DIGITAL|ECARE"),
            df_sac["METHOD"].str.contains("TELEFONO|MEDIOS|SMS|VISITA|CARTA|FAX|PROVEEDORES")
            ]
choicelist = ["DIGITAL", "TRADICIONAL"]
df_sac["CHANNEL"] = np.select(condlist, choicelist, default="OTHER")

# Generate features for contact channel
condlist = [df_sac["METHOD"].str.contains("IN|WHATS|SMS|WEB|VISITA|IVR"),
            df_sac["METHOD"].str.contains("MEDIOS") & ~(df_sac["CATEGORY_ID"] == "01"),
            df_sac["METHOD"].str.contains("OUT"),
            (df_sac["METHOD"].str.contains("MEDIOS")) & (df_sac["CATEGORY_ID"] == "01")
            ]
choicelist = ["IN", "IN", "OUT", "OUT"]
df_sac["CHANNEL_DIR"] = np.select(condlist, choicelist, default="OTHER")

# Format date vars
df_sac["FECHA_CREATE"] = df_sac["FECHA_CREATE"].dt.strftime("%Y%m%d")


In [17]:
df_sac.head()

,CUSTOMER_ID,FECHA_CREATE,CATEGORY,SUB_CATEGORY,SECOND_SUB_CATEGORY,METHOD,CATEGORY_ID,SUB_CATEGORY_ID,SECOND_SUB_CATEGORY_ID,CATEGORY_COMBINED,SUB_CATEGORY_COMBINED,CHANNEL,CHANNEL_DIR
0,144472,20180531,12-Optimizacion de Contactos,01-No Entiende/Error Factura,003-Valores facturados/Detalle/ Dise?o,VISITA,12,01,003,1201,1201003,TRADICIONAL,IN
1,144472,20180507,12-Optimizacion de Contactos,08-Otros,044-Cambio datos de la cuenta/forma de pago,VISITA,12,08,044,1208,1208044,TRADICIONAL,IN
2,144472,20180507,04-Finanzas,01-Cobranzas,003-Cancela metodo de pago,FAX,04,01,003,0401,0401003,TRADICIONAL,OTHER
3,145360,20180224,12-Optimizacion de Contactos,03-Sin Se?al,021-Fallas origen de transmision/sistemas-apoyo,SMS_IN,12,03,021,1203,1203021,TRADICIONAL,IN
4,145360,20180224,12-Optimizacion de Contactos,03-Sin Se?al,021-Fallas origen de transmision/sistemas-apoyo,TELEFONO_IN,12,03,021,1203,1203021,TRADICIONAL,IN


In [24]:
df_sac.loc[df_sac.CATEGORY_COMBINED == '0801'].groupby(['CATEGORY','SUB_CATEGORY']).size()

CATEGORY                   SUB_CATEGORY               
08-Retencion del servicio  01-Solicitud de Cancelacion    3167
dtype: int64

In [18]:
df_list = []
# Vars to dummy
log.info("Creating dummies...")
to_dummy = dict(zip(categories_dict.keys(),
                    ["N_CONT_CAT", "N_CONT_SUBCAT", "N_CONT_CH_DIR", "N_CONT_CH"]))
for var, var_name in to_dummy.items():
    log.info(f"----- {var}")
    df_list.append(impute_and_group_categorical_variables(df=df_sac,
                                                          vars_to_groupby=vars_to_group_by,
                                                          var_to_count=var,
                                                          var_to_count_name=var_name,
                                                          values=categories_dict[var]))
df_sac_dummies = reduce(
    lambda left, right: pd.merge(left, right, on=vars_to_group_by, how="outer", validate="1:1"), df_list)

# Calculate past variables
df_sac_dummies_past = add_relative_calculate_past(df_sac_dummies,
                                                  date_col="FECHA_CREATE",
                                                  id_cols=["CUSTOMER_ID"],
                                                  start_date=start_date,
                                                  end_date=date,
                                                  periods=past_periods,
                                                  agg=[np.nansum],
                                                  period_freq="D")

# Add date variables
df_sac_dummies_past["DATE_EXP"] = period_to_load
df_sac_dummies_past["DATE_CALC"] = date

2021-01-19 18:40:34,189 - aa_engine_pkg.assets.utils.utilities - INFO - Creating dummies...
2021-01-19 18:40:34,190 - aa_engine_pkg.assets.utils.utilities - INFO - ----- CATEGORY_COMBINED
['0605']
2021-01-19 18:40:37,727 - aa_engine_pkg.assets.utils.utilities - INFO - ----- SUB_CATEGORY_COMBINED
[]
2021-01-19 18:40:40,722 - aa_engine_pkg.assets.utils.utilities - INFO - ----- CHANNEL_DIR
2021-01-19 18:40:41,527 - aa_engine_pkg.assets.utils.utilities - INFO - ----- CHANNEL
2021-01-19 18:40:43,713 - aa_engine_pkg.assets.utils.utilities - INFO - Adding relative date between 20171219 and 20180605
2021-01-19 18:40:44,272 - aa_engine_pkg.assets.utils.utilities - INFO - Expanding table for period: 7
2021-01-19 18:40:44,680 - aa_engine_pkg.assets.utils.utilities - INFO - Expanding table for period: 28
2021-01-19 18:40:45,512 - aa_engine_pkg.assets.utils.utilities - INFO - Expanding table for period: 84
2021-01-19 18:40:46,983 - aa_engine_pkg.assets.utils.utilities - INFO - Expanding table for p

In [19]:
df_sac_dummies_past

,CUSTOMER_ID,N_CONT_SUBCAT_1206032_nansum_7,N_CONT_SUBCAT_1203018_nansum_7,N_CONT_CAT_1101_nansum_7,N_CONT_SUBCAT_0108006_nansum_7,N_CONT_SUBCAT_1202048_nansum_7,N_CONT_CH_DIGITAL_nansum_7,N_CONT_CAT_1003_nansum_7,N_CONT_SUBCAT_1207042_nansum_7,N_CONT_SUBCAT_0702007_nansum_7,...,N_CONT_CAT_1203_nansum_168,N_CONT_SUBCAT_1205029_nansum_168,N_CONT_CAT_0403_nansum_168,N_CONT_CAT_0805_nansum_168,N_CONT_SUBCAT_1203019_nansum_168,N_CONT_CAT_1206_nansum_168,N_CONT_CH_DIR_OTHER_nansum_168,N_CONT_CAT_0206_nansum_168,DATE_EXP,DATE_CALC
0,8031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,0,0,201805,20180605
1,8069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,1,0,0,0,0,201805,20180605
2,8176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,201805,20180605
3,8677,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,201805,20180605
4,8686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,0,0,0,0,0,201805,20180605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92384,53964888,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,201805,20180605
92385,53964986,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,201805,20180605
92386,53965981,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,201805,20180605
92387,53966010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,201805,20180605


In [ ]:

def create_master_servicioalcliente(servicioalcliente: SQLPartitionedDataSet,
                                    cliente_activo: pd.DataFrame,
                                    parameters: Dict,
                                    date: str) -> pd.DataFrame:
    """Creates mastertable with call center calls features for one period of data
    Parameters
    ----------
    servicioalcliente:
        dataset defined in ``catalog.yml`` with raw data information related to call center calls
    cliente_activo:
        dataset defined in ``catalog.yml`` with raw data information related to active clients at EoP
    date:
        period to process
    parameters:
        set of project parameters defined in ``parameters.yml``
    Returns
    -------
    pd.DataFrame
        mastertable with call center calls features for one period
    """

    # Initialize logger
    log = initialize_logger()

    write_to_parquet = parameters["write_to_parquet"]
    table_name = "servicioalcliente"
    overwrite = parameters["masters"][table_name]["overwrite"]

    # Check if table was already created
    files = get_mastertable_paths_by_period(parameters=parameters, period=date)
    match = [str(file) for file in files if table_name in file]

    if len(match) > 0 and overwrite is False:
        # If table is found, read parquet:
        log.info(f"Reading {match[0]} table")
        df_sac_dummies_past = pd.read_parquet(match[0], engine="pyarrow")

    else:
        log.info("Creating master table servicio al cliente")

        # Read parameters
        lookback_days = parameters["masters"]["global"]["look_back_days"]
        start_date = (pd.to_datetime(date) - timedelta(days=lookback_days)).strftime("%Y%m%d")

        # Calculate period to load for active clients
        log.info("Loading table clientes")
        period_to_load = get_previous_month(date)
        df_clientes = cliente_activo

        log.info("Loading table servicioalcliente")
        df_sac = servicioalcliente.filter_by(date=[start_date,
                                                   date]).drop_duplicates()

        df_sac = pd.merge(
            df_clientes,
            df_sac,
            on=["CUSTOMER_ID"],
            validate="1:m"
        )

        # Fill in rows with blank CATEGORY 
        mask_blank_category = df_sac.CATEGORY.isna()
        df_sac.loc[mask_blank_category, "CATEGORY"] = df_sac.loc[mask_blank_category, "SUB_CATEGORY"]
        df_sac.loc[mask_blank_category, "SUB_CATEGORY"] = df_sac.loc[mask_blank_category, "SECOND_SUB_CATEGORY"]
        df_sac.loc[mask_blank_category, "SECOND_SUB_CATEGORY"] = np.NaN

        # Format variables
        log.info("Extracting category codes...")
        vars_to_dummy = []
        for var in string_vars:
            log.info(f"--- {var}")
            if var == "METHOD":
                format_string_variable(df_sac, var)
            else:
                new_name = var + "_ID"
                vars_to_dummy.append(new_name)
                df_sac[new_name] = df_sac[var].str.extract(r"([a-zA-Z]*[\d]*)[ ]*\-")

        # Fill Second_sub_category_id 
        mask_ID_na = df_sac.SECOND_SUB_CATEGORY_ID.isna()
        condlist = [df_sac[mask_ID_na].SECOND_SUB_CATEGORY == "Contactado",
                    df_sac[mask_ID_na].SECOND_SUB_CATEGORY == "No contactado"]
        choicelist = ["C", "NC"]
        df_sac.loc[mask_ID_na, "SECOND_SUB_CATEGORY_ID"] = np.select(condlist, choicelist)

        # Create compound key for category of contact
        df_sac["CATEGORY_COMBINED"] = df_sac["CATEGORY_ID"] + df_sac["SUB_CATEGORY_ID"]
        df_sac["SUB_CATEGORY_COMBINED"] = df_sac["CATEGORY_ID"] + df_sac["SUB_CATEGORY_ID"] + df_sac[
            "SECOND_SUB_CATEGORY_ID"]

        # Generate features for contact channel
        condlist = [df_sac["METHOD"].str.contains("WEB|WHATS|SOCIAL|MAIL|CHAT|DIGITAL|ECARE"),
                    df_sac["METHOD"].str.contains("TELEFONO|MEDIOS|SMS|VISITA|CARTA|FAX|PROVEEDORES")
                    ]
        choicelist = ["DIGITAL", "TRADICIONAL"]
        df_sac["CHANNEL"] = np.select(condlist, choicelist, default="OTHER")

        # Generate features for contact channel
        condlist = [df_sac["METHOD"].str.contains("IN|WHATS|SMS|WEB|VISITA|IVR"),
                    df_sac["METHOD"].str.contains("MEDIOS") & ~(df_sac["CATEGORY_ID"] == "01"),
                    df_sac["METHOD"].str.contains("OUT"),
                    (df_sac["METHOD"].str.contains("MEDIOS")) & (df_sac["CATEGORY_ID"] == "01")
                    ]
        choicelist = ["IN", "IN", "OUT", "OUT"]
        df_sac["CHANNEL_DIR"] = np.select(condlist, choicelist, default="OTHER")

        # Format date vars
        df_sac["FECHA_CREATE"] = df_sac["FECHA_CREATE"].dt.strftime("%Y%m%d")

        df_list = []
        # Vars to dummy
        log.info("Creating dummies...")
        to_dummy = dict(zip(categories_dict.keys(),
                            ["N_CONT_CAT", "N_CONT_SUBCAT", "N_CONT_CH_DIR", "N_CONT_CH"]))
        for var, var_name in to_dummy.items():
            log.info(f"----- {var}")
            df_list.append(impute_and_group_categorical_variables(df=df_sac,
                                                                  vars_to_groupby=vars_to_group_by,
                                                                  var_to_count=var,
                                                                  var_to_count_name=var_name,
                                                                  values=categories_dict[var]))
        df_sac_dummies = reduce(
            lambda left, right: pd.merge(left, right, on=vars_to_group_by, how="outer", validate="1:1"), df_list)

        # Calculate past variables
        df_sac_dummies_past = add_relative_calculate_past(df_sac_dummies,
                                                          date_col="FECHA_CREATE",
                                                          id_cols=["CUSTOMER_ID"],
                                                          start_date=start_date,
                                                          end_date=date,
                                                          periods=past_periods,
                                                          agg=[np.nansum],
                                                          period_freq="D")

        # Add date variables
        df_sac_dummies_past["DATE_EXP"] = period_to_load
        df_sac_dummies_past["DATE_CALC"] = date

        # Change variable names 
        table_preffix = parameters["masters"]["servicioalcliente"]["table_preffix"]
        rename_table(df_sac_dummies_past,
                     preffix=table_preffix,
                     ids_to_exclude=id_cols)

        if write_to_parquet:
            file = f"{parameters['paths']['master_path']}master_{table_name}/master_{table_name}_{date}.parquet"
            df_sac_dummies_past.to_parquet(file, engine="pyarrow")

        log.info(f"Exporting {df_sac_dummies_past.shape[0]} rows and {df_sac_dummies_past.shape[1]} columns")

    return df_sac_dummies_past